In [12]:
import hashlib
import os
import random

from PIL import Image, ImageDraw, ImageFont


def calculate_image_size(text, img_font):
    temp_image = Image.new('RGB', (1, 1), color=(255, 255, 255))
    temp_draw = ImageDraw.Draw(temp_image)

    text_bbox = temp_draw.textbbox((0, 0), text, font=img_font)

    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    return text_width, text_height


def draw_text(text, img_font, background=(255, 255, 255)):
    padding = img_font.size // 2
    text_width, text_height = calculate_image_size(text, img_font)
    image = Image.new('RGB', (text_width + padding, text_height + padding), color=background)
    draw = ImageDraw.Draw(image)
    draw.text((0, -img_font.size / 10), text, font=img_font, fill=(0, 0, 0))
    return image


def apply_aging_effect(image, aging_factor=(0.6, 0.6, 0.01)):
    image = image.convert('L')
    black_factor, gray_factor, white_factor = aging_factor

    pixel_data = list(image.getdata())
    for i in range(len(pixel_data)):
        pixel_value = pixel_data[i]

        if pixel_value < 85:
            if random.random() > black_factor:
                continue
            new_value = pixel_value + random.randint(0, 20) * 10  # увеличиваем яркость 
        elif pixel_value < 170:
            if random.random() > gray_factor:
                continue
            new_value = pixel_value + random.randint(-20, 20) * 10  # меняем яркость 
        else:
            if random.random() > white_factor:
                continue
            new_value = pixel_value + random.randint(-20, 0) * 10  # уменьшаем яркость

        pixel_data[i] = new_value

    image.putdata(pixel_data)
    return image


def generate_file_name(text, img_font):
    text_md5 = hashlib.md5(text.encode('utf-8')).hexdigest()
    font_name = img_font.path.split('/')[-1].split('.')[0]
    font_md5 = hashlib.md5(font_name.encode('utf-8')).hexdigest()
    return f'{font_md5}_{text_md5}', font_name, text_md5


def generate_image(text, img_font, aging_factor=(0.5, 0.1, 0.005), directory='../fonts'):
    file_name, font_name, text_name = generate_file_name(text, img_font)
    if os.path.exists(os.path.join(directory, f'{file_name}.png')):
        return

    image = draw_text(text, img_font)
    image = apply_aging_effect(image, aging_factor)

    prefix_dir = ''
    os.makedirs(os.path.join(directory, prefix_dir), exist_ok=True)

    image.save(os.path.join(directory, prefix_dir, f'{file_name}.png'))

    with open(os.path.join(directory, prefix_dir, f'{file_name}.gt.txt'), 'w') as f:
        f.write(text)

In [21]:
FONT_SIZE = 20

TEXT_FILE = os.path.join('../data/tesstrain/kbd/data/output/freq_100000_oshamaho.txt')
FONT_DIR = '../data/tesstrain/kbd/fonts'
FONT_TEST_TEXT = 'Хъэндыркъуакъуэ щIакъуэр, КъуанщIэ лъакъуэм щIокъур.'

with open(TEXT_FILE, 'r') as f:
    lines = f.read().split('\n')

fonts_files = [f'{FONT_DIR}/{font}' for font in sorted(os.listdir(FONT_DIR)) if font.endswith('.ttf')]
fonts = []
for font_f in fonts_files:
    try:
        img_font = ImageFont.truetype(font_f, FONT_SIZE)
        fonts.append(img_font)
    except OSError:
        print(f'Error loading font {font_f}')
        continue


In [22]:
from concurrent.futures import ThreadPoolExecutor
import concurrent
from tqdm import tqdm

par_factor = 4  # Количество параллельных задач

OUTPUT_DIR = '/Users/panagoa/tesstrain_data/freq_100k_oshamaho_new_font'
os.makedirs(OUTPUT_DIR, exist_ok=True)

TEXT_LINES_MAX_COUNT = 1000000
text_lines = list(set([line for line in lines]))
random.shuffle(text_lines)
text_lines = text_lines[:TEXT_LINES_MAX_COUNT]

with tqdm(total=len(fonts) * len(text_lines)) as pbar, ThreadPoolExecutor(max_workers=par_factor) as executor:
    futures = [
        executor.submit(generate_image, text_line, font, directory=OUTPUT_DIR)
        for font in fonts for text_line in text_lines
    ]
    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()
            pbar.update(1)
        except Exception as e:
            print(f"Ошибка: {e}")

100%|██████████| 3300000/3300000 [3:04:41<00:00, 297.78it/s]  
